In [1]:
from pyspark.sql import SparkSession, DataFrame

spark = SparkSession.builder.appName('SparkByExamples.com').config('spark.driver.bindAddress','localhost').config("spark.ui.port","4051").config("spark.driver.memory","3g").getOrCreate()

23/02/12 10:03:11 WARN Utils: Your hostname, Chaturvedi_PC resolves to a loopback address: 127.0.1.1; using 172.27.64.6 instead (on interface eth0)
23/02/12 10:03:11 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
23/02/12 10:03:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [25]:
from pyspark.sql import functions as f

t20_df_with_boundaries = spark.read.parquet("processed_output/delivery_parquet")\
    .where(f.col('match_type') == 'T20') \
    .where(f.col('dt') > '2023-01-15') \
    .withColumn('is_boundary', f.when((f.col('batter_runs') >= 4) & (f.col("batter_runs") < 6), 1).otherwise(0)) \
    .withColumn('is_six', f.when(f.col("batter_runs") >= 6, 1).otherwise(0)) \
    .withColumn('is_dismissed', f.when(f.col("wicket_player_id") == f.col("batter_id"), 1).otherwise(0))

In [26]:
batter_relevant_dimensions = ["match_id", "dt", "venue_name", "batter_name", "batter_id"] 
batter_relevant_metrics = ["batter_runs","is_dismissed", "is_boundary", "is_six"]
t20_batter_delivery_df = t20_df_with_boundaries.select(batter_relevant_dimensions+batter_relevant_metrics)

In [27]:
from pyspark.sql.functions import sum, count
t20_batter_match_df  = t20_batter_delivery_df.groupBy(batter_relevant_dimensions) \
    .agg(
        sum("batter_runs").alias("batter_run_sum"),
        sum("is_dismissed").alias("dismissals"),
        sum("is_boundary").alias("boundary_count"),
        sum("is_six").alias("six_count"),
        count("batter_id").alias("deliveries")
    )